In [1]:
import pandas as pd
import data_fetching as df
from datetime import datetime
from orders import order_query
import os

In [2]:
# Variables for store
shop_url = os.environ.get("SHOP_URL", "")
api_version = "2023-04"
private_app_password = os.environ.get("PRIVATE_APP_PASSWORD", "")

In [3]:
client = df.shopify_client(shop_url, api_version, private_app_password)
client

In [4]:
bulk_query = df.bulk_operation(order_query)
bulk = client.execute(bulk_query)
bulk

'{"data":{"bulkOperationRunQuery":{"bulkOperation":{"id":"gid://shopify/BulkOperation/4123373371650","status":"CREATED"},"userErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":1000.0,"currentlyAvailable":990,"restoreRate":50.0}}}}'

In [5]:
status = df.bulk_status(client, bulk)
status

{'data': {'node': {'id': 'gid://shopify/BulkOperation/4123373371650',
   'status': 'COMPLETED',
   'errorCode': None,
   'createdAt': '2024-01-15T04:18:45Z',
   'completedAt': '2024-01-15T04:18:47Z',
   'objectCount': '211',
   'url': 'https://storage.googleapis.com/shopify-tiers-assets-prod-us-east1/hd2892ms2hwve4z3qk6mi18ujrmy?GoogleAccessId=assets-us-prod%40shopify-tiers.iam.gserviceaccount.com&Expires=1705897127&Signature=Iv9FW6TPG8kQDDkfBzDjak4X7geHd97uFwpgtzCIB7sDolbOeyTMjjCHEcGzt7wBvlSagZiYFJQkVwB8ZC4l0Xowp%2Biqh%2FuL3%2FUlteXm5ZIUBBDw1M8D4tPARSoTN9ZEnQeUYgrZfuQU%2BKdDrRLxp7yAAXE5ak3GCix0gaQS7WwUS2tGfTCGnjSHYEB7heN6V1p69KR%2F%2F5y8zQMLNHRdmQyksAQL3mdTP5W5NWr8WqeSoZjCgxMFRHg3H6dtV69EUsQ8FjgB6x8xOGr6I%2BSE0A3OyhB5aOf2C3jun0Hl7seAzSKIcTdc%2FzTKPlSnDQCOZ5bQaXVsnmRCj7woleO2ug%3D%3D&response-content-disposition=attachment%3B+filename%3D%22bulk-4123373371650.jsonl%22%3B+filename%2A%3DUTF-8%27%27bulk-4123373371650.jsonl&response-content-type=application%2Fjsonl',
   'partialDataUrl': No

In [6]:
orders = df.get_bulk_data(status)
orders[0]

{'id': 'gid://shopify/Order/5497601818882',
 'name': '#1001',
 'email': 'egnition_sample_47@egnition.com',
 'displayFinancialStatus': 'PENDING',
 'displayFulfillmentStatus': 'UNFULFILLED',
 'customerAcceptsMarketing': False,
 'currencyCode': 'VND',
 'subtotalPrice': '0',
 'discountCode': None,
 'note': None,
 'phone': None,
 'app': {'id': 'gid://shopify/App/1608003', 'name': 'Simple Sample Data'},
 'totalTax': '0',
 'totalPrice': '0',
 'createdAt': '2024-01-08T16:33:08Z',
 'totalDiscountsSet': {'shopMoney': {'amount': '0.0'}},
 'shippingLine': None,
 'billingAddress': {'address1': 'P.O. Box 879, 8424 Eget Street',
  'address2': None,
  'city': 'Bremerhaven',
  'province': None,
  'countryCode': 'BD',
  'zip': 'N27 2TJ',
  'provinceCode': None},
 'channelInformation': None,
 'shippingAddress': {'address1': 'P.O. Box 879, 8424 Eget Street',
  'address2': None,
  'city': 'Bremerhaven',
  'province': None,
  'countryCode': 'BD',
  'zip': 'N27 2TJ',
  'provinceCode': None}}

In [7]:
from data_validation import order_data_checking, line_item_data_checking, shipping_line_data_checking, \
                            fulfillment_data_checking, customer_data_checking, note_attributes_data_checking, \
                            tax_line_data_checking, assign_value
data = []
for order in orders:
    order_name, order_email, order_financial_status, order_fulfillment_status, order_buyer_accepts_marketing, \
    order_currency, order_subtotal_price, order_total_tax, order_total_price, order_total_discounts,  \
    order_created_at, order_note, order_cancelled_at, order_payment_method, order_refunds, order_payment_reference, \
    order_id, order_tags, order_source, order_phone, order_duties, order_payment_terms_name, order_discount_code = order_data_checking(order)
    
    discount_code = ""
    
    shipping_price, shipping_method = "", ""
    
    billing_first_name, billing_last_name, billing_address1, billing_address2, billing_company, \
    billing_city, billing_zip, billing_province, billing_country, billing_phone, billing_province_name \
    = "", "", "", "", "", "" "", "", "", "", "", ""

    shipping_first_name, shipping_last_name, shipping_address1, shipping_address2, shipping_company, \
    shipping_city, shipping_zip, shipping_province, shipping_country, shipping_phone, shipping_province_name \
    = "", "", "", "", "", "", "", "", "", "", ""
    
    note_attributes = ""
    
    fulfillment_at = ""

    line_item_quantity, line_item_name, line_item_price, line_item_sku, \
    line_item_requires_shipping, line_item_taxable, line_item_fulfillment_status, \
    line_item_vendor, line_item_discount = "", "", "", "", "", "", "", "", ""
    
    tax_line_name, tax_line_rate, tax_line_value = "", "", ""
    
    order_data = {
        "Name": order_name["name"] if order_name != "" else "",
        "Email": order_email["email"] if order_email != "" else "",
        "Financial Status": order_financial_status["displayFinancialStatus"].lower() if order_financial_status != "" else "",
        "Fulfillment Status": order_fulfillment_status["displayFulfillmentStatus"].lower() if order_fulfillment_status != "" else "",
        # "Fulfilled at": datetime.fromisoformat(fulfillment_at.created_at).strftime("%Y-%m-%d %H:%M:%S%z") if fulfillment_at != "" else "",
        "Accepts Marketing": order_buyer_accepts_marketing["customerAcceptsMarketing"] if order_buyer_accepts_marketing != "" else "",
        "Currency": order_currency["currencyCode"] if order_currency != "" else "",
        "Subtotal": order_subtotal_price["subtotalPrice"] if order_subtotal_price != "" else "",
        # "Shipping": shipping_price.price if shipping_price != "" else "",
        # "Taxes": order_total_tax.total_tax if order_total_tax != "" else "",
        # "Total": order_total_price.total_price if order_total_price != "" else "",
        # "Discount Code": discount_code.code if discount_code != "" else "",
        # "Discount Amount": order_total_discounts.total_discounts if order_total_discounts != "" else "",
        # "Shipping Method": shipping_method.code if shipping_method != "" else "",
        # "Created At": datetime.fromisoformat(order_created_at.created_at).strftime("%Y-%m-%d %H:%M:%S%z") if order_created_at else "",
        # "Lineitem quantity": line_item_quantity.quantity if line_item_quantity != "" else "",
        # "Lineitem name": line_item_name.name if line_item_quantity != "" else "",
        # "Lineitem price": line_item_price.price if line_item_price != "" else "",
        # "Lineitem compare at price": "Not yet", # Todo
        # "Lineitem sku": line_item_sku.sku if line_item_sku != "" else "",
        # "Lineitem requires shipping": line_item_requires_shipping.requires_shipping if line_item_requires_shipping != "" else "",
        # "Lineitem taxable": line_item_taxable.taxable if line_item_taxable != "" else "",
        # "Lineitem fulfillment status": line_item_fulfillment_status.fulfillment_status if line_item_fulfillment_status != "" else "",
        # "Billing Name": billing_first_name.first_name if billing_first_name != "" else "" + " " + billing_last_name.last_name if shipping_last_name != "" else "",
        # "Billing Street": billing_address1.address1 if billing_address1 != "" else "" + " " + billing_address2.address2 if billing_address2 != "" else "",
        # "Billing Address1": billing_address1.address1 if billing_address1 != "" else "",
        # "Billing Address2": billing_address2.address2 if billing_address2 != "" else "",
        # "Billing Company": billing_company.company if billing_company != "" else "",
        # "Billing City": billing_city.city if billing_city != "" else "",
        # "Billing Zip": billing_zip.zip if billing_zip != "" else "",
        # "Billing Province": billing_province.province_code if billing_province != "" else "",
        # "Billing Country": billing_country.country_code if billing_country != "" else "",
        # "Billing Phone": billing_phone.phone if billing_phone != "" else "",
        # "Shipping Name": shipping_first_name.first_name if shipping_first_name != "" else ""  + " " + shipping_last_name.last_name if shipping_last_name != "" else "",
        # "Shipping Street": shipping_address1.address1 if shipping_address1 != "" else ""  + " " + shipping_address2.address2 if shipping_address1 != "" else "",
        # "Shipping Address1": shipping_address1.address1 if shipping_address1 != "" else "",
        # "Shipping Address2": shipping_address2.address2 if shipping_address1 != "" else "",
        # "Shipping Company": shipping_company.company if shipping_company != "" else "",
        # "Shipping City": shipping_city.city if shipping_city != "" else "",
        # "Shipping Zip": shipping_zip.zip if shipping_zip != "" else "",
        # "Shipping Province": shipping_province.province_code if shipping_province != "" else "",
        # "Shipping Country": shipping_country.country_code if shipping_country != "" else "",
        # "Shipping Phone": shipping_phone.phone if shipping_phone != "" else "",
        # "Note": order_note.note if order_note != "" else "",
        # "Note Attributes": note_attributes,
        # "Cancelled At" : order_cancelled_at.cancelled_at,
        # "Payment Method" : order_payment_method.payment_gateway_names if order_payment_method != "" else "",
        # "Payment Reference": order_payment_reference.reference if order_payment_reference != "" else "",
        # "Refunded Amount": order_refunds.refunds if order_refunds != "" else "",
        # "Vendor": line_item_vendor.vendor if line_item_vendor != "" else "",
        # "Id": order_id.id if order_id.id != "" else "",
        # "Tags": order_tags.tags if order_tags != "" else "",
        # "Risk Level": "Not yet", # Todo
        # "Source": order_source.source_name if order_source != "" else "",
        # "Phone": order_phone.phone if order_phone != "" else "",
        # "Receipt Number": "Not yet", # Todo
        # "Duties": order_duties.current_total_duties_set if order_duties != "" else "",
        # "Billing Province Name": billing_province_name.province if billing_province != "" else "",
        # "Shipping Province Name": shipping_province_name.province if shipping_province_name != "" else "",
        # "Payment Terms Name": order_payment_terms_name.payment_terms if order_payment_terms_name != "" else "",
        # "Next Payment Due At": "Not yet", # Todo
        # "Payment References": order_payment_reference.reference if order_payment_reference != "" else "",
    }
    
    # for fulfillment in order.fulfillments:
    #     fulfillment_at = fulfillment_data_checking(fulfillment)
        
    #     # datetime.fromisoformat(fulfillment_data_checking(fulfillment)).strftime("%Y-%m-%d %H:%M:%S%z")
    
    # for discount in order.discount_applications:        
    #     discount_code = discount_data_checking(discount)
    
    # for shipping_line in order.shipping_lines:
    #     shipping_price, shipping_method = shipping_line_data_checking(shipping_line)
    
    # for line_item in order.line_items:
    #     line_item_quantity, line_item_name, line_item_price, line_item_sku, \
    #     line_item_requires_shipping, line_item_taxable, line_item_fulfillment_status, \
    #     line_item_vendor, line_item_discount = line_item_data_checking(line_item)
    
    
    # for tax_line_index, tax_line in enumerate(order.tax_lines, start=1):
    #     tax_line_name, tax_line_rate ,tax_line_value = tax_line_data_checking(tax_line)
    #     order_data[f"Tax {tax_line_index} Name"] = f"{tax_line_name.title} {format(float(tax_line_value.rate) * 100, '.2f')} %" if tax_line_name != "" and tax_line_value != "" else ""
    #     order_data[f"Tax {tax_line_index} Value"] = tax_line_value.price if tax_line_value != "" else ""

    
    # for billing in [order.billing_address]:
    #     billing_first_name, billing_last_name, billing_address1, billing_address2, billing_company, \
    #     billing_city, billing_zip, billing_province, billing_country, billing_phone, billing_province_name \
    #     = customer_data_checking(billing)
    
    # for shipping in [order.shipping_address]:
    #     shipping_first_name, shipping_last_name, shipping_address1, shipping_address2, shipping_company, \
    #     shipping_city, shipping_zip, shipping_province, shipping_country, shipping_phone, shipping_province_name \
    #     = customer_data_checking(shipping)
    
    # for notes in order.note_attributes:
    #     note_attributes = note_attributes_data_checking(notes)
    
    
    data.append(order_data)

for d in data:
#     if d["Fulfillment Status"] is None:
#         d["Fulfilled at"] = ""
#         d["Fulfillment Status"] = "unfulfilled"
    if d["Accepts Marketing"] == False:
        d["Accepts Marketing"] = "no"
    if d["Accepts Marketing"] == True:
        d["Accepts Marketing"] = "yes"
#     if d["Lineitem fulfillment status"] is None:
#         d["Lineitem fulfillment status"] = "pending"
#     for method in d["Payment Method"]:
#         if method == "bogus":
#             d["Payment Method"] = "(for testing) Bogus Gateway"
#         elif method == "shopify_payments":
#             d["Payment Method"] = "Shopify Payments"
#         else:
#             d["Payment Method"] = ""
        

order_df  = pd.DataFrame(data)
order_df.fillna("", inplace=True)
# df["Refunded Amount"] = df["Refunded Amount"].map(assign_value)
order_df

,Name,Email,Financial Status,Fulfillment Status,Accepts Marketing,Currency,Subtotal
0,#1001,egnition_sample_47@egnition.com,pending,unfulfilled,no,VND,0
1,#1002,egnition_sample_47@egnition.com,paid,unfulfilled,no,VND,0
2,#1003,egnition_sample_27@egnition.com,pending,unfulfilled,no,VND,0
3,#1004,egnition_sample_47@egnition.com,pending,unfulfilled,no,VND,0
4,#1005,egnition_sample_47@egnition.com,pending,unfulfilled,no,VND,0
...,...,...,...,...,...,...,...
86,#1087,,paid,unfulfilled,no,VND,120
87,#1088,,paid,unfulfilled,no,VND,120
88,#1089,,paid,unfulfilled,no,VND,120
89,#1090,,paid,unfulfilled,no,VND,120
